# Epoching + TFRs for all subjects


*Created: 09/14/2024* \
*Updated: 11/21/2024* 


In [1]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import re
import datetime
import scipy
from ast import literal_eval



import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/sc/arion/projects/guLab/Alie/SWB/ephys_analysis/LFPAnalysis/')

In [4]:
from LFPAnalysis import lfp_preprocess_utils, sync_utils, analysis_utils, nlx_utils

In [5]:
# Specify root directory for un-archived data and results 
base_dir   = '/sc/arion/projects/guLab/Alie/SWB/ephys_analysis/'
anat_dir   = f'{base_dir}recon_labels/'
neural_dir = f'{base_dir}data/'
behav_dir  = f'{base_dir}behav/behav_data/'
raw_dir    = f'{base_dir}behav/raw_data/'
pt_path    = f'{base_dir}behav/subj_pt_data/'
save_path  = f'{base_dir}epoch_data/'

date = datetime.date.today().strftime('%m%d%Y')
print(date)

11212024


In [6]:
# sys.path.append(f'{behav_dir}scripts/')
script_dir = '/hpc/users/finka03/swb_ephys_analysis/scripts/'
sys.path.append(f'{script_dir}behav/')

from behav_utils import *
from swb_subj_behav import * 

## Gather All Subj Info

In [20]:
##### MS050 + MS028 CANNOT BE INCLUDED - THEY DID NOT DO THE MOOD TASK

subj_ids = list(pd.read_excel(f'{base_dir}subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID)
subj_ids.remove('MS050')
subj_ids.remove('MS028')
n_subj = len(subj_ids)
subj_ids


['MS002',
 'MS003',
 'MS009',
 'MS011',
 'MS015',
 'MS016',
 'MS017',
 'MS019',
 'MS022',
 'MS024',
 'MS025',
 'MS026',
 'MS027',
 'MS029',
 'MS030',
 'MS033',
 'MS035',
 'MS041',
 'MS043',
 'MS048',
 'DA8',
 'DA023',
 'DA026',
 'DA037',
 'DA039']

In [21]:
### load ROI reref master 
roi_reref_labels_master_df = pd.read_csv(glob(f'{base_dir}results/roi_info/roi_reref_labels_master.csv')[0])
roi_reref_labels_master_df = roi_reref_labels_master_df.drop(columns=['Unnamed: 0'])
roi_reref_labels_master_df = roi_reref_labels_master_df[roi_reref_labels_master_df.subj_id.isin(subj_ids)]
roi_reref_labels_master_df



,subj_id,reref_ch_names,ch_label4roi,ch_type4roi,loc4roi,mni_x,mni_y,mni_z,hemi,roi
0,MS002,lacas1-lacas2,lacas1,anode,left cingulate gyrus d,-6.382462,37.158688,-3.130044,l,acc
1,MS002,lacas2-lacas3,lacas2,anode,left cingulate gyrus e,-6.368174,38.606223,2.270621,l,acc
2,MS002,lacas3-lacas4,lacas3,anode,left cingulate gyrus f,-6.390079,39.941566,7.640265,l,acc
3,MS002,lacas4-lacas5,lacas4,anode,left cingulate gyrus f,-6.914519,41.546899,12.993427,l,acc
4,MS002,lacas5-lacas6,lacas5,anode,left cingulate gyrus g,-6.933604,42.745784,18.267675,l,acc
...,...,...,...,...,...,...,...,...,...,...
1875,DA039,rtp6-rtp7,rtp6,anode,right superior middle temporal pole d,40.532268,7.354525,-33.253436,r,temporal pole
1876,DA039,rtp7-rtp8,rtp7,anode,right superior middle temporal pole d,43.911226,7.341369,-31.915085,r,temporal pole
1877,DA039,rsgcc6-rsgcc7,rsgcc7,cathode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc
1878,DA039,rsgcc7-rsgcc8,rsgcc7,anode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc


In [22]:
anat_info_all_subj = {f'{subj_id}': pd.read_csv(f'{anat_dir}{subj_id}_labels.csv') for subj_id in subj_ids}


In [23]:
task_controls = pd.read_csv('/sc/arion/projects/guLab/Alie/SWB/ephys_analysis/subj_info/swb_task_controls.csv')
# task_controls.head()
# max_iti = np.max(task_controls.ITI)
# max_iti

# Load behav + mood data

In [24]:
# load task_dfs into master list 
raw_behav = [pd.read_csv(f'{behav_dir}{subj_id}_task_df.csv') for subj_id in subj_ids]
all_behav,beh_drops = format_all_behav(raw_behav, return_drops=True, drop_bads=False, drop_bads_t1=False,
                     norm=True,norm_type='zscore',pt_dir=pt_path)
all_behav

,subj_id,bdi,bdi_thresh,Round,TrialNum,RT,TrialOnset,ChoiceOnset,DecisionOnset,FeedbackOnset,...,risk_raw,risk_t1_raw,P_Safe_raw,P_Safe_t1_raw,loss_raw,loss_t1_raw,util_HighBet_raw,util_HighBet_t1_raw,Unnamed: 0_raw,Unnamed: 0_t1_raw
0,MS002,14,low,1,25.0,2.059852,513.380590,513.390239,515.450091,515.457173,...,0.819173,0.819173,0.897969,0.638282,2.373595,2.373595,0.030942,-0.995865,0,1.0
1,MS002,14,low,2,117.0,1.954564,522.640856,522.641563,524.596127,526.627092,...,0.819173,0.819173,0.638282,0.341225,2.373595,2.373595,-0.999406,1.493454,1,2.0
2,MS002,14,low,3,79.0,1.583462,531.174799,531.175599,532.759061,534.780269,...,0.819173,0.819173,0.341225,0.714422,2.373595,2.373595,1.498244,0.893763,2,3.0
3,MS002,14,low,4,42.0,2.491611,545.592613,545.593355,548.084966,548.092333,...,0.819173,0.819173,0.714422,0.483386,2.373595,2.373595,0.896547,0.945208,3,4.0
4,MS002,14,low,5,85.0,1.768936,555.337336,555.345720,557.114656,559.135069,...,0.819173,0.819173,0.483386,0.882421,2.373595,2.373595,0.948163,0.261760,4,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,DA039,22,high,146,79.0,1.079701,2259.827656,2259.828749,2260.908450,2262.926195,...,1.187095,1.187095,0.209864,0.723269,0.864195,0.864195,1.505088,-0.112975,145,146.0
3746,DA039,22,high,147,30.0,1.837272,2267.502359,2267.534059,2269.371331,2269.377701,...,1.187095,1.187095,0.723269,0.509159,0.864195,0.864195,-0.133242,-0.373925,146,147.0
3747,DA039,22,high,148,13.0,4.030006,2282.349445,2282.350662,2286.380667,2286.389886,...,1.187095,1.187095,0.509159,0.585308,0.864195,0.864195,-0.382021,-0.360674,147,148.0
3748,DA039,22,high,149,18.0,3.167144,2293.040983,2293.042042,2296.209186,2296.218136,...,1.187095,1.187095,0.585308,0.884229,0.864195,0.864195,-0.369389,-0.830554,148,149.0


In [27]:
### load mood info all subj 
raw_mood = [pd.read_csv(f'{behav_dir}{subj_id}_mood_df.csv') for subj_id in subj_ids]
all_mood, mood_drops = format_all_mood(raw_mood, all_behav, return_drops=True, drop_bads=False)


In [28]:
all_mood


,subj_id,bdi,bdi_thresh,Round,Rating,RatingOnset,RT,Round_t1_idx,Round_t2_idx,Round_t3_idx,...,rpe_t3,cf_t1,cf_t2,cf_t3,cpe_t1,cpe_t2,cpe_t3,mood_epoch_len,next_round_start,norm_mood
0,MS002,14,low,4,70.0,539.343131,5.210698,3,2,1,...,-0.017374,0.551367,-0.307083,0.759848,1.461373,-0.892581,-0.795008,5.310698,545.592613,0.277664
1,MS002,14,low,7,70.0,572.673634,3.332630,6,5,4,...,-0.017374,-1.288170,0.674003,-0.920262,1.241834,0.802754,0.875934,3.432630,577.049490,0.277664
2,MS002,14,low,10,69.0,605.109393,3.499831,9,8,7,...,-1.720166,0.060824,-0.307083,-0.674991,-1.441430,0.266101,-1.185300,3.599831,609.642231,0.134538
3,MS002,14,low,13,71.0,638.791836,3.884104,12,11,10,...,-1.103047,1.164546,0.306096,0.428731,-1.197497,0.680788,-0.465698,3.984104,643.713750,0.420789
4,MS002,14,low,16,71.0,671.977904,4.250371,15,14,13,...,-0.017374,0.060824,0.796639,0.060824,1.729700,-0.831598,-0.831598,4.350371,677.264040,0.420789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,DA039,22,high,139,45.0,2176.828649,2.673696,138,137,136,...,-0.887088,NaN,-0.479399,0.915739,NaN,0.314126,-0.772921,2.773696,2180.513505,-0.524890
1246,DA039,22,high,142,45.0,2211.211948,2.961470,141,140,139,...,0.046736,0.760724,-3.362685,-0.014353,-0.652138,1.835992,-0.289789,3.061470,2215.197736,-0.524890
1247,DA039,22,high,145,55.0,2246.799223,3.347751,144,143,142,...,NaN,-1.967547,0.760724,NaN,1.111294,1.884306,NaN,3.447751,2251.180979,0.330135
1248,DA039,22,high,148,68.0,2276.912826,4.403968,147,146,145,...,-1.152672,-2.339584,0.605708,0.760724,1.763523,1.497800,-0.652138,4.503968,2282.349445,1.441667


In [29]:
all_mood.to_csv(f'{base_dir}behav/all_mood_{date}.csv')
all_mood.to_csv(f'{base_dir}behav/results_data/all_mood.csv')



# Create and Save Epoched Data for all subjects

In [14]:
#### Set global vars for all subj 


baseline_ev  = 'RatingOnset' # fixation cross baseline epoch
analysis_evs = ['MoodChoiceOnset']

# dictionary of event ids and times - repeated from above
evs = {'RatingOnset': [-1.0, 0.0], #ITI is always at least one second + 10ms buffer for wavelet padding (no horizontal smearing)
       'MoodChoiceOnset': [-1.0,0.0] } # 3 sec following CPE outcome reveal - timestamps are made manually above

In [15]:
# IED removal requires that we set some parameters for IED detection. 
# 1. peak_thresh: how many stds should they exceed the baseline by? 
# 2. closeness_thresh: how close should they be allowed to be (in sec) to other candidate IEDs? 
# 3. width_thresh: how wide should they have to be (in sec)?

# Defaults:
IED_args = {'peak_thresh':5,
           'closeness_thresh':0.25, 
           'width_thresh':0.2}

In [16]:
# Define spectrogram parameters for morlet wavelets 

freqs = np.logspace(*np.log10([2, 200]), num=30)
n_cycles = np.floor(np.logspace(*np.log10([3, 10]), num=30))


In [17]:

for subj_id in subj_ids: 
    
    print(f'start epoching {subj_id}')
    
    # load photodiode data
    diode_info_fname  = f'{neural_dir}diode_info/{subj_id}_diode_info.npy'
    slope,offset = np.load(diode_info_fname)
    # load behav data 
    mood_df = all_mood[all_mood.subj_id == subj_id]

    # make dictionary to store epoch data 
    epochs_all_evs = {f'{x}': np.nan for x in evs}
    
    for event in evs.keys():
        ### make epochs should now find IEDs, artifact times, and save info as df in load_path directory for every ev
        # Make the epochs. 
        ev_epochs = lfp_preprocess_utils.make_epochs(load_path=f'{neural_dir}{subj_id}/bp_ref_ieeg.fif', 
                                                     slope=slope, offset=offset,
                                                     behav_name=event, behav_times=mood_df[event].values,
                                                     ev_start_s=evs[event][0], ev_end_s=evs[event][1],
                                                     buf_s = 1, IED_args=IED_args,
                                                     downsamp_factor=None,  
                                                     detrend=0, 
                                                     #### new - constant detrend by DC offset https://mne.tools/stable/generated/mne.Epochs.html
                                                     save_path = save_path, ### SWB PIPELINE ADDITION - ADD SAVE PATH & SUBJ ID FOR DATA ORGANIZATION
                                                     subj_id=subj_id)

        epochs_all_evs[event] = ev_epochs
        
        ### save epoched data - make new data saving path
        ev_path = f'{save_path}{event}/'
        os.makedirs(ev_path,exist_ok=True)
        
        ### save baseline tfr data         
        ev_epochs.save(f'{ev_path}/{subj_id}_{event}_epochs.fif', overwrite=True)    
        del ev_epochs 
        
    #### COMPUTE BASELINE TFRs FOR ONE SUBJ
    
    # baseline epoch dictionary 
    epochs_all_baseline = {} 

    # baseline event = TrialOnset epochs 
    baseline_epochs_reref = epochs_all_evs[baseline_ev]

    # dropping bad channels from baseline epochs 
    baseline_epochs_reref = baseline_epochs_reref.drop_channels(baseline_epochs_reref.info['bads'])

    # compute baseline tfr 
    temp_pow  = mne.time_frequency.tfr_morlet(baseline_epochs_reref, freqs=freqs, n_cycles=n_cycles, 
                                              picks=baseline_epochs_reref.ch_names, # now doesn't include bad ch
                                              use_fft=True, n_jobs=-1, output='power',
                                              return_itc=False, average=False) # returns pow (not ITC) for all epochs

    temp_pow.crop(tmin=evs[baseline_ev][0], tmax=evs[baseline_ev][1]) # clipping epoch to remove buffer

    ### DROP IED/Artifacts from baseline! Remove bad data by naning out +/-100ms around bad timepoints (IEDs and artifacts)
    
    # THE following will now LOAD in dataframes that indicate IED and artifact time points in your data
    IED_df      = pd.read_csv(f'{save_path}{baseline_ev}/{subj_id}_{baseline_ev}_IED_df.csv') 
    artifact_df = pd.read_csv(f'{save_path}{baseline_ev}/{subj_id}_{baseline_ev}_artifact_df.csv') 

    # iterate through each channel, IED/artifact & nan tfr data +/-100ms from IED/artifact
    for ch_ix, ch_name in enumerate(baseline_epochs_reref.ch_names):
        
        # lists of IED/artifact timestamps from dfs for one channel (if any)
        ied_ev_list = IED_df[ch_name].dropna().index.tolist()
        artifact_ev_list = artifact_df[ch_name].dropna().index.tolist()

        # iterate through every IED timestamp in channel 
        for ev_ in ied_ev_list: 
            # iterate through values in IED df (literal eval = evaluates string as literal python data structures)
            for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
                # remove 100 ms before 
                ev_ix_start = np.max([0, np.floor((ied_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
                # remove 100 ms after 
                ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
                temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

        # iterate through every artifact timestamp in channel 
        for ev_ in artifact_ev_list: 
             # iterate through values in artifact df (literal eval = evaluates string as literal python data structures)
            for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
                # remove 100 ms before 
                ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
                # remove 100 ms after
                ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
                temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

    #### save baseline tfr
    temp_pow.save(f'{save_path}{baseline_ev}/{subj_id}_{baseline_ev}-tfr.h5', overwrite=True) 
    #### store baseline tfr data to dict
    epochs_all_baseline[baseline_ev] = temp_pow
    
    # delete clunky data (could rename temp_pow instead of double saving because there's only one baseline)
    del temp_pow,IED_df,artifact_df,baseline_epochs_reref 
    

    ##### COMPUTE TFR FOR ANALYSIS EPOCHS AND BASELINE TO BASELINE TFR DATA
        # repeat baselining procedure for every event in analysis_evs
        # remove +/-100ms around IED/artifact individually for each analysis epoch 
        # baseline analysis ev tfrs
        #### IGNORE BELOW FOR NOW
            # iteratively check for zscores still above 10 std, remove those timepoints, then recompute baselined tfr

    for event in analysis_evs: 
        
        print('Compute baselined TFR for epoch: ',event)
        
        # extract raw epoch data for analysis ev 
        event_epochs_reref = epochs_all_evs[event].copy()
        # dropping bad channels from baseline epochs --- don't think there's any bad ch in fif
        event_epochs_reref = event_epochs_reref.drop_channels(event_epochs_reref.info['bads'])
        ch_names = event_epochs_reref.ch_names

        # compute raw event TFR from raw epoched data
        temp_pow = mne.time_frequency.tfr_morlet(event_epochs_reref, freqs=freqs, n_cycles=n_cycles, 
                                                 picks=event_epochs_reref.ch_names, 
                                                 use_fft=True, n_jobs=-1, output='power', 
                                                 return_itc=False, average=False)
        # crop 1.0 sec buffer from analysis epoch 
        temp_pow.crop(tmin=evs[event][0], tmax=evs[event][1])

        #### DROP IEDs/ARTIFACTS FROM EVENT EPOCH repeat IED/artifact removal procedure used for baseline epochs, but with analysis event IED/artifact times

        # load IED/artifact dfs specific to event in analysis evs
        IED_df      = pd.read_csv(f'{save_path}{event}/{subj_id}_{event}_IED_df.csv') 
        artifact_df = pd.read_csv(f'{save_path}{event}/{subj_id}_{event}_artifact_df.csv') 

        # iterate through each channel, IED/artifact & nan tfr data +/-100ms from IED/artifact
        for ch_ix, ch_name in enumerate(event_epochs_reref.ch_names): 
            ied_ev_list = IED_df[ch_name].dropna().index.tolist()
            artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
            for ev_ in ied_ev_list: 
                for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
                    # remove 100 ms before 
                    ev_ix_start = np.max([0, np.floor((ied_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
                    # remove 100 ms after
                    ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
                    temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
            for ev_ in artifact_ev_list: 
                for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
                    # remove 100 ms before 
                    ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
                    # remove 100 ms after
                    ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
                    temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

        
        ### COMPUTE NORMALIZED TFR FOR EVENT EPOCH
        baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data,
                                                                       include_epoch_in_baseline=False,  
                                                                       baseline_mne=epochs_all_baseline[baseline_ev].data,  
                                                                       mode='zscore', ev_axis=0, elec_axis=1, 
                                                                       freq_axis=2, time_axis=3)

                
        #### remove extreme power values (z>10) remaining after IED/artifact removal 
        # iterative process - baseline correct event TFR, remove z > 10, re-baseline until no z>10 or reached max iters

        # Let's nan out absurd z-scores (10 std above baseline???) that escaped our artifact detection, noise removal, and baselining
        absurdity_threshold = 10 # zscore threshold for removal
        max_iter = 10 # max number of iterations for while loop 
        large_z_flag=True # flag that indicates whether large zs are present, when all z<10 = False in while loop

        iteration = 0 # intialize iter counter
        while (large_z_flag==True) & (iteration<max_iter): # while z>10 still in tfr or under max iter num 
            print(f'baseline z-score iteration # {iteration}')
            # Baseline by all the baseline periods in the session
            baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data,include_epoch_in_baseline=False, 
                                                                                   baseline_mne=epochs_all_baseline[baseline_ev].data,  
                                                                                   mode='zscore', ev_axis=0, elec_axis=1, 
                                                                                   freq_axis=2, time_axis=3)

            large_z_mask = np.where(np.abs(baseline_corrected_power)>absurdity_threshold)
            if large_z_mask[0].shape[0] == 0:
                # no more large z
                large_z_flag = False
            else:
                # NaN it out in the event of interest prior to re-running the baseline z-score to prevent
                # contamination of all z's
                temp_pow.data[large_z_mask] = np.nan

            iteration +=1

        # input baseline corrected pow into mne EpochsTFR object once all z>10 removed or iteratively removed 10x 
        zpow = mne.time_frequency.EpochsTFR(event_epochs_reref.info, baseline_corrected_power,
                                            temp_pow.times, freqs)

        # save data (#conda install -c conda-forge h5io must install to save tfr data)
        zpow.save(f'{save_path}{event}/{subj_id}_{event}-tfr.h5', overwrite=True) 
        
        print(f'finished epoching {subj_id}')
        
        del zpow, temp_pow, event_epochs_reref
    
    del epochs_all_evs, epochs_all_baseline


start epoching MS002
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/bp_ref_ieeg.fif...
    Range : 0 ... 1083499 =      0.000 ...  2166.998 secs
Ready.
Reading 0 ... 1083499  =      0.000 ...  2166.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)
- Filter length: 265 samples (0.530 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    2.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  89 out of  96 | elapsed:    2.2s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    2.3s finished


Used Annotations descriptions: ['RatingOnset']
Not setting metadata
50 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 50 events and 1501 original time points ...
0 bad epochs dropped
Opening raw data file /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/bp_ref_ieeg.fif...
    Range : 0 ... 1083499 =      0.000 ...  2166.998 secs
Ready.
Reading 0 ... 1083499  =      0.000 ...  2166.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 80.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 90.00 Hz)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    1.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  89 out of  96 | elapsed:    2.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    2.4s finished


Used Annotations descriptions: ['MoodChoiceOnset']
Not setting metadata
50 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 50 events and 1501 original time points ...
0 bad epochs dropped
Overwriting existing file.
Overwriting existing file.
NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").


...

Used Annotations descriptions: ['MoodChoiceOnset']
Not setting metadata
36 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 36 events and 1501 original time points ...
0 bad epochs dropped
NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:    2.4s finished


Compute baselined TFR for epoch:  MoodChoiceOnset
NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:    1.8s finished


baseline z-score iteration # 0
baseline z-score iteration # 1
finished epoching DA039


In [18]:
subj_ids

['MS002',
 'MS003',
 'MS009',
 'MS011',
 'MS015',
 'MS016',
 'MS017',
 'MS019',
 'MS022',
 'MS024',
 'MS025',
 'MS026',
 'MS027',
 'MS028',
 'MS029',
 'MS030',
 'MS033',
 'MS035',
 'MS041',
 'MS043',
 'MS048',
 'MS050',
 'DA8',
 'DA023',
 'DA026',
 'DA037',
 'DA039']

In [48]:
# # for every subject - 


# for subj_id in subj_ids: 
#     print(f'start epoching {subj_id}')
    
#     # load photodiode data
#     diode_info_fname  = f'{neural_dir}diode_info/{subj_id}_diode_info.npy'
#     slope,offset = np.load(diode_info_fname)
#     # load behav data 
#     mood_df = all_mood[all_mood.subj_id == subj_id]
#     # update behav_df to have a cpe timestamp 
        
#     epochs_all_evs = {f'{x}': np.nan for x in evs}
    
#     ### make epochs should now find IEDs, artifact times, and save info as df in load_path directory for every ev

#     for event in evs.keys():
#         # Make the epochs. 
#         ev_epochs = lfp_preprocess_utils.make_epochs(load_path=f'{neural_dir}{subj_id}/bp_ref_ieeg.fif', 
#                                                      slope=slope, offset=offset,
#                                                      behav_name=event, behav_times=mood_df[event].values,
#                                                      ev_start_s=evs[event][0], ev_end_s=evs[event][1],
#                                                      buf_s = 1, IED_args=IED_args,
#                                                      downsamp_factor=None,  
#                                                      detrend=0) #### new - constant detrend by DC offset https://mne.tools/stable/generated/mne.Epochs.html

#         epochs_all_evs[event] = ev_epochs
#         ### save epoched data 
#         ev_epochs.save(f'{neural_dir}{subj_id}/{subj_id}_{event}_epochs.fif', overwrite=True)
    
    
#     #### COMPUTE BASELINE TFRs FOR ONE SUBJ
#     # baseline epoch dictionary 
#     epochs_all_baseline = {} 

#     # baseline event = TrialOnset epochs 
#     baseline_epochs_reref = epochs_all_evs[baseline_ev]

#     # dropping bad channels from baseline epochs 
#     baseline_epochs_reref = baseline_epochs_reref.drop_channels(baseline_epochs_reref.info['bads'])

#     # compute baseline tfr 
#     temp_pow  = mne.time_frequency.tfr_morlet(baseline_epochs_reref, freqs=freqs, n_cycles=n_cycles, 
#                                               picks=baseline_epochs_reref.ch_names, # now doesn't include bad ch
#                                               use_fft=True, n_jobs=-1, output='power',
#                                               return_itc=False, average=False) # returns pow (not ITC) for all epochs

#     temp_pow.crop(tmin=evs[baseline_ev][0], tmax=evs[baseline_ev][1]) # clipping epoch to remove buffer

#     # Remove bad data by naning out +/-100ms around bad timepoints (IEDs and artifacts)
#     # THE following will now LOAD in dataframes that indicate IED and artifact time points in your data
#     IED_df = pd.read_csv(f'{neural_dir}{subj_id}/{baseline_ev}_IED_df.csv') 
#     artifact_df = pd.read_csv(f'{neural_dir}{subj_id}/{baseline_ev}_artifact_df.csv') 

#     # iterate through each channel, IED/artifact & nan tfr data +/-100ms from IED/artifact
#     for ch_ix, ch_name in enumerate(baseline_epochs_reref.ch_names):
#         # lists of IED/artifact timestamps from dfs for one channel (if any)
#         ied_ev_list = IED_df[ch_name].dropna().index.tolist()
#         artifact_ev_list = artifact_df[ch_name].dropna().index.tolist()

#         # iterate through every IED timestamp in channel 
#         for ev_ in ied_ev_list: 
#             # iterate through values in IED df (literal eval = evaluates string as literal python data structures)
#             for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
#                 # remove 100 ms before 
#                 ev_ix_start = np.max([0, np.floor((ied_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                 # remove 100 ms after 
#                 ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                 temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

#         # iterate through every artifact timestamp in channel 
#         for ev_ in artifact_ev_list: 
#              # iterate through values in artifact df (literal eval = evaluates string as literal python data structures)
#             for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
#                 # remove 100 ms before 
#                 ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                 # remove 100 ms after
#                 ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                 temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

#     epochs_all_baseline[baseline_ev] = temp_pow


#     ##### COMPUTE TFR FOR ANALYSIS EPOCHS AND BASELINE TO BASELINE TFR DATA

#         # repeat baselining procedure for every event in analysis_evs
#         # remove +/-100ms around IED/artifact individually for each analysis epoch 
#         # baseline analysis ev tfrs
#         # iteratively check for zscores still above 10 std, remove those timepoints, then recompute baselined tfr


#     power_epochs = {} # keys = analysis evs 

#     for event in analysis_evs: 
#         print('Compute baselined TFR for epoch: ',event)

#         # extract raw epoch data for analysis ev 
#         event_epochs_reref = epochs_all_evs[event].copy()
#         # dropping bad channels from baseline epochs 
#         event_epochs_reref = event_epochs_reref.drop_channels(event_epochs_reref.info['bads'])
#         ch_names = event_epochs_reref.ch_names

#         # compute TFR from raw epoched data
#         temp_pow = mne.time_frequency.tfr_morlet(event_epochs_reref, freqs=freqs, n_cycles=n_cycles, 
#                                                  picks=event_epochs_reref.ch_names, 
#                                                  use_fft=True, n_jobs=-1, output='power', 
#                                                  return_itc=False, average=False)
#         # crop 1.0 sec buffer from analysis epoch 
#         temp_pow.crop(tmin=evs[event][0], tmax=evs[event][1])

#         ### repeat IED/artifact removal procedure used for baseline epochs, but with analysis event IED/artifact times

#         # load IED/artifact dfs specific to event in analysis evs
#         IED_df = pd.read_csv(f'{neural_dir}{subj_id}/{event}_IED_df.csv') 
#         artifact_df = pd.read_csv(f'{neural_dir}{subj_id}/{event}_artifact_df.csv') 

#         # iterate through each channel, IED/artifact & nan tfr data +/-100ms from IED/artifact
#         for ch_ix, ch_name in enumerate(event_epochs_reref.ch_names): 
#             ied_ev_list = IED_df[ch_name].dropna().index.tolist()
#             artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
#             for ev_ in ied_ev_list: 
#                 for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
#                     # remove 100 ms before 
#                     ev_ix_start = np.max([0, np.floor((ied_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                     # remove 100 ms after
#                     ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                     temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
#             for ev_ in artifact_ev_list: 
#                 for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
#                     # remove 100 ms before 
#                     ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                     # remove 100 ms after
#                     ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                     temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan

#         ##### remove extreme power values (z>10) remaining after IED/artifact removal 
#         # iterative process - baseline correct event TFR, remove z > 10, re-baseline until no z>10 or reached max iters


#         # Let's nan out absurd z-scores (10 std above baseline???) that escaped our artifact detection, noise removal, and baselining
#         absurdity_threshold = 10 # zscore threshold for removal
#         max_iter = 10 # max number of iterations for while loop 
#         large_z_flag=True # flag that indicates whether large zs are present, when all z<10 = False in while loop

#         iteration = 0 # intialize iter counter
#         while (large_z_flag==True) & (iteration<max_iter): # while z>10 still in tfr or under max iter num 
#             print(f'baseline z-score iteration # {iteration}')
#             # Baseline by all the baseline periods in the session
#             baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data, 
#                                   baseline_mne=epochs_all_baseline[baseline_ev].data,  
#                                   mode='zscore', ev_axis=0, elec_axis=1, freq_axis=2, time_axis=3)

#             large_z_mask = np.where(baseline_corrected_power>absurdity_threshold)
#             if large_z_mask[0].shape[0] == 0:
#                 # no more large z
#                 large_z_flag = False
#             else:
#                 # NaN it out in the event of interest prior to re-running the baseline z-score to prevent
#                 # contamination of all z's
#                 temp_pow.data[large_z_mask] = np.nan

#             iteration +=1

#         # input baseline corrected pow into mne EpochsTFR object once all z>10 removed or iteratively removed 10x 
#         zpow = mne.time_frequency.EpochsTFR(event_epochs_reref.info, baseline_corrected_power,
#                                             temp_pow.times, freqs)

#         # add metadata to baseline corrected TFR event epochs object 
#         zpow.metadata = event_epochs_reref.metadata

#         # save data (#conda install -c conda-forge h5io must install to save tfr data)
#         zpow.save(f'{neural_dir}{subj_id}/{subj_id}_{event}-tfr.h5', overwrite=True) 

#         # store event zpow epochs in dict for all analysis evs 
#         power_epochs[event] = zpow
        
#         print(f'finished epoching {subj_id}')
        
#     del power_epochs, zpow, temp_pow, large_z_mask, event_epochs_reref, ev_epochs, epochs_all_evs, epochs_all_baseline






# Plot TFRs 
- For each analysis event, plot the mean TFR across all epochs for all channels
- Save plots
- Visually inspect TFRs to see if further data cleaning is necessary

In [ ]:
# ####add directory for TFRs
# tfr_dir = f'{base_dir}ephys_analysis/figs/single_elec_tfrs/{subj_id}/'

# # print date of TFR plotting 
# date = datetime.date.today().strftime('%m%d%Y')
# print(date, offset) # print date, double check current slope/offset


In [ ]:
# #### for each analysis event - plot one TFR averaged across all epochs for every electrode & save 

# for event in analysis_evs: 
#     print('Plot TFR for epoch: ',event)
#     save_path = f'{tfr_dir}{event}/'
#     os.makedirs(save_path,exist_ok = True) #added so you don't have to manually make folders
    
#     ev_data = power_epochs[event].copy() # tfr plot data - epochs x num ch x freqs x times 
#     times = ev_data.times.copy() # same for all elec
    
#     for ch in ev_data.ch_names:
#         # single ch tfr 
#         plot_data = np.nanmean(np.nanmean(ev_data.copy().pick_channels([ch]).data, axis=0), axis=0) 
        
#         fig, ax = plt.subplots(1, 1, figsize=(6, 4),dpi=300)
        
#         im = ax.imshow(plot_data,interpolation='Bicubic',
#                        aspect='auto', origin='lower', cmap='RdBu_r')
    

#         ax.set(xlabel='Time (s)', ylabel='Frequency (Hz)',title=f'{subj_id}_{ch}_{event}_{date}',
#                yticks=np.arange(0,len(freqs))[::5], 
#                yticklabels=freqs[::5].astype(int), 
#                xticks = np.arange(plot_data.shape[1])[::250],
#                   xticklabels= times[::250])
        
#         fig.colorbar(im, ax=ax)

#         plt.savefig(f'{save_path}{subj_id}_{ch}_{event}_{date}.png', dpi=300)
# #         bbox_inches=None, pad_inches=0.1,
# #         facecolor='auto', edgecolor='auto',backend=None)
#         plt.close()

    

In [ ]:
# # from salman - faster code

# def compute_and_baseline_tfr(baseline_event, task_events, freqs, n_cycles, load_path, save_path,
#                             IED_artifact_thresh=True, uncaptured_z_thresh=True, output='save'):
    
#     """
    
#     """
    
    
#     baseline_name = list(baseline_event.keys())[0]
    
#     # load baseline epochs
#     baseline_epochs_reref = mne.read_epochs(f'{load_path}/{baseline_name}-epo.fif', preload=True)
    
#     # compute TFR
#     baseline_power  = mne.time_frequency.tfr_morlet(baseline_epochs_reref, 
#                                           freqs=freqs, 
#                                           n_cycles=n_cycles, 
#                                           picks=baseline_epochs_reref.ch_names,
#                                           use_fft=True, 
#                                           n_jobs=-1, 
#                                           output='power', 
#                                           return_itc=False, 
#                                           average=False)
    

#     # Crop the data to the appropriate 
#     baseline_power.crop(tmin=baseline_event[baseline_name][0], 
#                         tmax=baseline_event[baseline_name][1])
    
    
#     if IED_artifact_thresh:
#         # NAN out the bad data
#         # THE following will now LOAD in dataframes that indicate IED and artifact time points in your data
#         IED_df = pd.read_csv(f'{load_path}/{baseline_name}_IED_df.csv') 
#         artifact_df = pd.read_csv(f'{load_path}/{baseline_name}_artifact_df.csv') 


#         # Now, let's iterate through each channel, and each ied/artifact, and NaN 100 ms before and after these timepoints
#         for ch_ix, ch_name in enumerate(baseline_epochs_reref.ch_names): 
#             ied_ev_list = IED_df[ch_name].dropna().index.tolist()
#             artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
#             for ev_ in ied_ev_list: 
#                 for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
#                     # remove 100 ms before 
#                     ev_ix_start = np.max([0, np.floor((ied_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                     # remove 100 ms after 
#                     ev_ix_end = np.min([baseline_power.data.shape[-1], np.ceil((ied_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                     baseline_power.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
#             for ev_ in artifact_ev_list: 
#                 for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
#                     # remove 100 ms before 
#                     ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                     # remove 100 ms after
#                     ev_ix_end = np.min([baseline_power.data.shape[-1], np.ceil((artifact_ + 0.1) * baseline_epochs_reref.info['sfreq'])]).astype(int)
#                     baseline_power.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
    
#     # remove epochs from memory
#     del baseline_epochs_reref
    
#     # Now we will deal with the task events of interest   
    
#     # make the output path for the z-scored TFRs
#     if not os.path.exists(save_path):
#         os.makedirs(save_path)
        
#     for event in task_events.keys():
        
#         event_epochs_reref = mne.read_epochs(f'{load_path}/{event}-epo.fif', preload=True)
                    
#         temp_pow = mne.time_frequency.tfr_morlet(event_epochs_reref, 
#                                                  freqs=freqs, 
#                                                  n_cycles=n_cycles,
#                                                  picks=event_epochs_reref.ch_names, 
#                                                  use_fft=True, 
#                                                  n_jobs=-1, 
#                                                  output='power', 
#                                                  return_itc=False, 
#                                                  average=False)
    
#         temp_pow.crop(tmin=task_events[event][0], tmax=task_events[event][1])
        
#         if IED_artifact_thresh:
#             # NAN out the bad data
#             # THE following will now LOAD in dataframes that indicate IED and artifact time points in your data
#             IED_df = pd.read_csv(f'{load_path}/{event}_IED_df.csv') 
#             artifact_df = pd.read_csv(f'{load_path}/{event}_artifact_df.csv') 

#             # Now, let's iterate through each channel, and each ied/artifact, and NaN 100 ms before and after these timepoints
#             for ch_ix, ch_name in enumerate(event_epochs_reref.ch_names): 
#                 ied_ev_list = IED_df[ch_name].dropna().index.tolist()
#                 artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
#                 for ev_ in ied_ev_list: 
#                     for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
#                         # remove 100 ms before 
#                         ev_ix_start = np.max([0, np.floor((ied_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                         # remove 100 ms after
#                         ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((ied_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                         temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
#                 for ev_ in artifact_ev_list: 
#                     for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
#                         # remove 100 ms before 
#                         ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                         # remove 100 ms after
#                         ev_ix_end = np.min([temp_pow.data.shape[-1], np.ceil((artifact_ + 0.1) * event_epochs_reref.info['sfreq'])]).astype(int)
#                         temp_pow.data[ev_, ch_ix, :, ev_ix_start:ev_ix_end] = np.nan    
    
#         # Compute first pass of baseline
#         baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data, include_epoch_in_baseline=False, 
#                                                                        baseline_mne=baseline_power.data, 
#                                                                        mode='zscore', ev_axis=0, elec_axis=1, freq_axis=2, time_axis=3)
            
#         # Let's iteratively nan out absurd z-scores (10 std above baseline???) that escaped our artifact detection, noise removal, and baselining
#         if uncaptured_z_thresh: 
#             absurdity_threshold = 10
#             max_iter = 10
#             large_z_flag=True 

#             iteration = 0

#             while (large_z_flag==True) & (iteration<max_iter): 
#                 print(f'baseline z-score iteration # {iteration}')
#                 # Baseline by all the baseline periods in the session
#                 baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data, include_epoch_in_baseline=False, 
#                                       baseline_mne=baseline_power.data, 
#                                       mode='zscore', ev_axis=0, elec_axis=1, freq_axis=2, time_axis=3)

#                 large_z_mask = np.where(np.abs(baseline_corrected_power)>absurdity_threshold)
#                 if large_z_mask[0].shape[0] == 0:
#                     # no more large z
#                     large_z_flag = False
#                 else:
#                     # NaN it out in the event of interest prior to re-running the baseline z-score to prevent
#                     # contamination of all z's
#                     temp_pow.data[large_z_mask] = np.nan

#                 iteration +=1
        
#         zpow = mne.time_frequency.EpochsTFR(event_epochs_reref.info, baseline_corrected_power, 
#                                     temp_pow.times, freqs)

#         zpow.metadata = event_epochs_reref.metadata
        
#         if output == 'save':
#             zpow.save(f'{save_path}/{event}-tfr.h5', overwrite=True)
#         elif output == 'return': 
#             return zpow 
#         elif output == 'both':
#             zpow.save(f'{save_path}/{event}-tfr.h5', overwrite=True)
#             return zpow 
        
# 12:32
# Usage example:
# # Explicitly define a list of analysis events and the baseline event. Should correspond to the dict

# baseline_event = {'baseline_start':[0, 0.75]}
# task_events = {'gamble_start':[-1.0, 0], 
#        'feedback_start': [0, 1.5]}

# day = 'Day1'

# # Define parameters for TFR: 
# freqs = np.logspace(*np.log10([2, 200]), num=30)
# n_cycles = np.floor(np.logspace(*np.log10([3, 10]), num=30))

# subj_id = 'MS012'

# compute_and_baseline_tfr(baseline_event, task_events, freqs, n_cycles, 
#                          load_path=f'{base_dir}/projects/guLab/Salman/EphysAnalyses/{subj_id}/neural/{day}', 
#                          save_path=f'{base_dir}/projects/guLab/Salman/EphysAnalyses/{subj_id}/scratch/TFR',
#                          IED_artifact_thresh=True, uncaptured_z_thresh=True, output='save')

# Get epoch timing

In [36]:
# next_behav_rounds = []
# next_round_starts = []
# next_round_iti    = []
# mood_round_starts = []
# prev_round_starts = []
# prev_round_iti    = []
# prev_round_ends   = []


# for subj_id in all_mood.subj_id.unique().tolist():
#     mood_df    = all_mood[all_mood.subj_id == subj_id].reset_index()
#     beh_df     = all_behav[all_behav.subj_id==subj_id].reset_index()
#     mood_round = mood_df.Round
#     next_round = mood_df.Round_t3_idx[1:]
#     iti_epochs = [task_controls[task_controls.Trial==round].ITI.values[0] for round in next_round]
    
#     prev_round = mood_df.Round_t1_idx[:-1]
#     prev_iti   = [task_controls[task_controls.Trial==round].ITI.values[0] for round in prev_round]
#     prev_ends  = mood_df.RatingOnset[:-1]-np.array(prev_iti)
    
#     mood_round_starts.append(mood_df.RatingOnset[:-1])
#     next_behav_rounds.append(next_round)
#     next_round_starts.append([beh_df.loc[beh_df.Round.astype(int)==rnd,'TrialOnset'].values[0] for rnd in next_round])
#     next_round_iti.append(iti_epochs)
#     prev_round_iti.append(prev_iti)
#     prev_round_ends.append(prev_ends)
    


In [26]:
# mood_epochs_end = []
# mood_behav_itis = []
# mood_timing_err = []
# mood_epochs_dur = []

# for ix,subj_id in enumerate(all_mood.subj_id.unique().tolist()):
#     mood_df     = all_mood[all_mood.subj_id == subj_id].reset_index()
#     beh_df      = all_behav[all_behav.subj_id==subj_id].reset_index()
#     mood_starts = mood_round_starts[ix]
#     beh_starts  = next_round_starts[ix]
#     iti_epochs  = next_round_iti[ix]
#     mood_rts    = mood_df.RT[:-1]
#     end_mood    = mood_starts+mood_rts+0.1
#     start_iti   = np.array(beh_starts)-np.array(iti_epochs)
#     mood_epochs_end.append(end_mood)
#     mood_behav_itis.append(start_iti)
#     mood_timing_err.append(start_iti-end_mood)
#     mood_epochs_dur.append(end_mood-mood_starts)
#     ### mood epochs ITI are all 1 sec!
    


In [27]:
# bad_mood_epochs = {}
# good_lens = []
# bad_lens  = []
# for ix,subj_id in enumerate(all_mood.subj_id.unique().tolist()):
#     epoch_lens = mood_epochs_dur[ix]
# #     print(subj_id, 'min epoch dur: ',np.min(epoch_lens),'max epoch dur: ',np.max(epoch_lens))
#     bad_mood_epochs[subj_id] = epoch_lens[epoch_lens<1].index
#     good_lens.append(epoch_lens[epoch_lens>=1])
#     bad_lens.append(epoch_lens[epoch_lens<1])
# #     print(subj_id, 'num epochs < 1sec ', len(epoch_lens[epoch_lens<1]))

In [28]:
# top5_epochs = []
# for ix,subj_id in enumerate(all_mood.subj_id.unique().tolist()):
#     print(good_lens[ix].sort_values(ascending=False)[:5])
#     top5_epochs.append(good_lens[ix].sort_values(ascending=False)[:5].values)

17    5.542328
11    5.317101
0     5.310698
29    5.102100
21    4.520483
dtype: float64
13    7.174198
6     6.676483
1     5.102575
23    5.070984
42    4.526620
dtype: float64
0    8.214453
4    7.259709
9    7.129213
1    6.892613
8    5.559149
dtype: float64
46    17.543819
12    13.143016
14     9.519730
43     8.556826
13     7.179213
dtype: float64
40    59.392014
12    15.433190
23    14.823507
29    14.762216
36    14.260199
dtype: float64
30    6.049961
13    5.963132
4     5.710379
7     5.129533
5     4.497239
dtype: float64
45    10.207823
38     7.575164
3      6.958037
6      6.950455
9      6.730176
dtype: float64
29    135.682643
36    130.042346
25     13.776958
3      13.766397
11     11.322839
dtype: float64
10    330.079428
41     12.305830
26     10.687746
0       9.490664
40      9.149162
dtype: float64
34    6.429771
30    6.364934
28    6.287827
1     6.251226
16    6.067334
dtype: float64
0     26.017828
2      9.687645
39     9.117747
12     7.909400
11    

In [29]:
# top5_epochs
##### drop >20 sec epoch lens 

[array([5.5423278, 5.3171015, 5.3106976, 5.1020999, 4.5204827]),
 array([7.1741981, 6.6764826, 5.1025746, 5.0709839, 4.5266202]),
 array([8.214453 , 7.2597092, 7.1292132, 6.8926128, 5.5591489]),
 array([17.5438194, 13.1430162,  9.51973  ,  8.5568261,  7.179213 ]),
 array([59.3920137, 15.4331899, 14.8235067, 14.7622161, 14.2601992]),
 array([6.0499614, 5.9631325, 5.7103788, 5.1295326, 4.4972391]),
 array([10.2078225,  7.5751636,  6.9580374,  6.9504552,  6.7301764]),
 array([135.682643 , 130.0423462,  13.7769578,  13.7663967,  11.322839 ]),
 array([330.0794281,  12.3058305,  10.6877462,   9.4906641,   9.1491621]),
 array([6.429771 , 6.3649342, 6.2878272, 6.2512259, 6.067334 ]),
 array([26.017828 ,  9.6876453,  9.1177468,  7.9093996,  7.7715932]),
 array([15.8830022, 13.3479898, 12.5737891, 11.6823982,  9.6874645]),
 array([7.6948789, 5.2481031, 5.0388214, 5.0030056, 4.9584489]),
 array([40.0865427, 38.3777853, 34.4632199, 15.4992833, 14.4765885]),
 array([12.714061 ,  9.8947742,  9.46904

In [30]:
# bottom5_epochs = []
# for ix,subj_id in enumerate(all_mood.subj_id.unique().tolist()):
#     print(good_lens[ix].sort_values(ascending=True)[:5])
#     bottom5_epochs.append(good_lens[ix].sort_values(ascending=True)[:5].values)

24    1.195020
27    1.289211
44    1.709886
45    1.858220
32    1.898677
dtype: float64
32    1.323477
34    1.642722
41    1.881581
30    2.149801
36    2.189441
dtype: float64
42    1.907153
48    1.917551
44    1.920518
46    1.985019
41    2.055968
dtype: float64
31    1.165874
6     1.602296
27    1.664193
26    1.740702
29    1.748370
dtype: float64
5     2.756534
19    3.637031
22    4.333957
20    4.552181
0     4.559313
dtype: float64
26    1.396996
31    1.427379
23    1.428601
25    1.459689
47    1.491830
dtype: float64
32    2.636046
36    2.689776
46    2.843062
29    2.942227
47    3.065692
dtype: float64
37    2.862558
41    3.382719
42    3.945193
10    4.087563
34    4.319370
dtype: float64
28    1.836435
2     3.019723
1     3.181366
32    3.272757
24    3.313495
dtype: float64
7     3.858700
6     4.200679
21    4.364853
3     4.483228
40    4.519811
dtype: float64
34    1.633030
17    2.232836
32    2.438497
40    2.773777
41    3.191697
dtype: float64
41    3.38